In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import classification_report
from abc import ABC


In [ ]:
class hateLabels(Dataset, ABC):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

class hateModel(ABC):
    def __init__(self, model_name='bert-base-uncased', lr=1e-5):
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name).to(self.device)
        self.optimizer = AdamW(self.model.parameters(), lr=lr)
        self.model.train()

    def tokenize(self, text):
        return self.tokenizer(text.tolist(), truncation=True, padding=True, max_length=128)

    def create_data_loader(self, encodings, labels, batch_size=16):
        dataset = hateLabels(encodings, labels)
        return DataLoader(dataset, batch_size=batch_size)

    def train(self, train_texts, train_labels, model_name='model.pt', val_texts=None, val_labels=None, epochs=5):
        train_encodings = self.tokenize(train_texts)
        train_loader = self.create_data_loader(train_encodings, train_labels)
        val_loader = None

        if val_texts is not None and val_labels is not None:
            val_encodings = self.tokenize(val_texts)
            val_loader = self.create_data_loader(val_encodings, val_labels)

        for epoch in range(epochs):
            print(f'Epoch {epoch+1}/{epochs}')
            print('-' * 10)

            train_loss = 0
            val_loss = 0

            # Train the model
            for batch in train_loader:
                batch = {k: v.to(self.device) for k, v in batch.items()}
                outputs = self.model(**batch)
                loss = outputs[0]
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                train_loss += loss.item()

            self.model.eval()

            # Validate the model
            if val_loader:
                for batch in val_loader:
                    with torch.no_grad():
                        batch = {k: v.to(self.device) for k, v in batch.items()}
                        outputs = self.model(**batch)
                        loss = outputs[0]
                        val_loss += loss.item()

                print(f'Train loss {train_loss / len(train_loader)}')
                print(f'Validation loss {val_loss / len(val_loader)}')
            else:
                print(f'Train loss {train_loss / len(train_loader)}')

            self.model.train()

        # torch.save(self.model.state_dict(), model_name)
        # To save:
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'loss': loss,},  model_name)

    def evaluate(self, test_texts, test_labels):
      self.model.eval()
      test_encodings = self.tokenize(test_texts)
      test_loader = self.create_data_loader(test_encodings, test_labels)
      predictions = []
      actuals = []

      with torch.no_grad():
          for batch in test_loader:
              batch = {k: v.to(self.device) for k, v in batch.items()}
              outputs = self.model(**batch)
              _, preds = torch.max(outputs.logits, dim=1)
              predictions.extend(preds.tolist())
              actuals.extend(batch['labels'].tolist())

      print(classification_report(actuals, predictions))



In [ ]:
# import  data for model 1
train_m1 = pd.read_csv('/content/train_m1.csv')
valid_m1 = pd.read_csv('/content/valid_m1.csv')
test_m1 = pd.read_csv('/content/test_m1.csv')

# import  data for model 2
train_m2 = pd.read_csv('/content/train_m2.csv')
valid_m2 = pd.read_csv('/content/valid_m2.csv')
test_m2 = pd.read_csv('/content/test_m2.csv')


In [ ]:
hate_model1 = hateModel()
hate_model1.train(train_m1['review'], train_m1['label'], 'model_1.pt',valid_m1['review'], valid_m1['label'])
hate_model1.evaluate(test_m1['review'], test_m1['label'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/5
----------
Train loss 0.5610729292449024
Validation loss 0.4674145282506943
Epoch 2/5
----------
Train loss 0.46602758460692945
Validation loss 0.3773753650188446
Epoch 3/5
----------
Train loss 0.36253470417836475
Validation loss 0.299565083861351
Epoch 4/5
----------
Train loss 0.24339015863554936
Validation loss 0.2604717476069927
Epoch 5/5
----------
Train loss 0.16901933113648926
Validation loss 0.2248575918376446
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       812
           1       0.94      0.95      0.95      1188

    accuracy                           0.94      2000
   macro avg       0.93      0.93      0.93      2000
weighted avg       0.93      0.94      0.93      2000



In [ ]:
hate_model2 = hateModel()
hate_model2.train(train_m2['review'], train_m2['label'], 'model_2.pt',valid_m2['review'], valid_m2['label'])
hate_model2.evaluate(test_m2['review'], test_m2['label'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/5
----------
Train loss 0.5566032426595688
Validation loss 0.6057542969310095
Epoch 2/5
----------
Train loss 0.4788188274741173
Validation loss 0.605188672031675
Epoch 3/5
----------
Train loss 0.39160174690485
Validation loss 0.6852821419163356
Epoch 4/5
----------
Train loss 0.2745631692767143
Validation loss 0.8466342342278314
Epoch 5/5
----------
Train loss 0.17097576160877942
Validation loss 1.1309309658550082
              precision    recall  f1-score   support

           0       0.55      0.49      0.52       362
           1       0.70      0.75      0.72       574

    accuracy                           0.65       936
   macro avg       0.63      0.62      0.62       936
weighted avg       0.64      0.65      0.65       936

